In [1]:
import numpy as np
import pandas as pd
import pickle
import sys
sys.path.insert(0, '..')
from InverseMfcc import InverseMfcc
import librosa
import matplotlib.pyplot as plt

def genSpectogram(y1, title1, y2, title2):
    D = np.abs(librosa.stft(y))**2
    S = librosa.feature.melspectrogram(S=D)
    
    # Passing through arguments to the Mel filters
    
    plt.subplot(2, 1, 1)    
    S = librosa.feature.melspectrogram(y=y1, sr=16000, n_mels=256)
    librosa.display.specshow(librosa.power_to_db(S,
                                                 ref=np.max),
                             y_axis='mel', fmax=16000,
                             x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title1)
    plt.tight_layout()
    
    plt.subplot(2, 1, 2)
    S = librosa.feature.melspectrogram(y=y2, sr=16000, n_mels=256)
    librosa.display.specshow(librosa.power_to_db(S,
                                                 ref=np.max),
                             y_axis='mel', fmax=16000,
                             x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title2)
    plt.tight_layout()

    plt.show()

# Fast Gradient Sign

#### One word

In [2]:
test_vs_pred_file10 = "one_word/preds_vs_labels_iters10.tsv"
tp10 = pd.read_table(test_vs_pred_file10)
tp10[tp10["same_diff"] == 'DIFF']

,filename,same_diff,pred_on_orig,pred_on_adv,target,num_pred_on_orig,num_pred_on_adv,num_target
0,above.wav,DIFF,iofe,above,above,10 16 7 6,2 3 16 23 6,2 3 16 23 6
1,bob.wav,DIFF,bob,bup,bob,3 16 3,3 22 17,3 16 3
2,reflection.wav,DIFF,reflect on,reflection,reflection,19 6 7 13 6 4 21 1 16 15,19 6 7 13 6 4 21 10 16 15,19 6 7 13 6 4 21 10 16 15
3,phenomenon.wav,DIFF,a nomenents,anomenons,phenomenon,2 1 15 16 14 6 15 6 15 21 20,2 15 16 14 6 15 16 15 20,17 9 6 15 16 14 6 15 16 15
4,token.wav,DIFF,token,hoken,token,21 16 12 6 15,9 16 12 6 15,21 16 12 6 15
6,imagine.wav,DIFF,magin,imadgined,imagine,14 2 8 10 15,10 14 2 5 8 10 15 6 5,10 14 2 8 10 15 6
7,miracle.wav,DIFF,americol,amierical,miracle,2 14 6 19 10 4 16 13,2 14 10 6 19 10 4 2 13,14 10 19 2 4 13 6
8,arch.wav,DIFF,ardshu,arhep,arch,2 19 5 20 9 22,2 19 9 6 17,2 19 4 9
9,plastic.wav,DIFF,plasti,plastic,plastic,17 13 2 20 21 10,17 13 2 20 21 10 4,17 13 2 20 21 10 4
10,universal.wav,DIFF,tht universa,the universa,universal,21 9 21 1 22 15 10 23 6 19 20 2,21 9 6 1 22 15 10 23 6 19 20 2,22 15 10 23 6 19 20 2 13


The original audio:

In [3]:
index = 15

adv1_ex_file = open("one_word/adv_x_iters10.npy")
orig1_ex_file = open("one_word/orig_x_iters10.npy")

orig1 = pickle.load(orig1_ex_file)
adv1 = pickle.load(adv1_ex_file)

#Original - change index here to change recording
filename = '../audios/' + tp10.ix[index]['filename']
print filename

y, sr = librosa.load(filename, mono=True, sr=None)
y = y[::3]
imfcc = InverseMfcc(y, 16000)

# Load NN outputs
orig_mfcc = orig1[0][index].transpose([1, 0])
adv_mfcc = adv1[0][index].transpose([1, 0])
zero_padding = ~np.all(orig_mfcc == 0, axis=0)
orig_mfcc = orig_mfcc[:, zero_padding]
adv_mfcc = adv_mfcc[:, zero_padding]

imfcc.play_original()

../audios/five.wav


The reconstructed original file:

In [4]:
imfcc.transform(orig_mfcc)
imfcc.play_transformed()
#imfcc.get_mfcc()

The reconstructed adversarial example:

In [5]:
imfcc.transform(adv_mfcc)
imfcc.play_transformed()
#imfcc.get_mfcc()

#imfcc.get_mfcc()
#genSpectogram(imfcc.recon)


Improving reconstruction quality with more MFCCs:

In [6]:
imfcc.modify_parameters(num_mfcc=150)
imfcc.transform()
imfcc.modify_parameters(num_mfcc=20)
imfcc.play_transformed()

#### Sentence

In [7]:
advv10_ex_file = open("vctk/adv_x_iters10.npy")
origv10_ex_file = open("vctk/orig_x_iters10.npy")
origv10 = pickle.load(origv10_ex_file)
advv10 = pickle.load(advv10_ex_file)

test_vs_predv10_file = "vctk/preds_vs_labels_iters10.tsv"
tpv10 = pd.read_table(test_vs_predv10_file)

tpv10[tpv10['same_diff'] == 'DIFF'][:14]

,filename,same_diff,pred_on_orig,pred_on_adv,target,num_pred_on_orig,num_pred_on_adv,num_target
0,p271_088.wav,DIFF,it jus shows the arogance of labour an the lib...,it just shows the arigance of labour and the l...,it just shows the arrogance of labour and the ...,10 21 1 11 22 20 1 20 9 16 24 20 1 21 9 6 1 2 ...,10 21 1 11 22 20 21 1 20 9 16 24 20 1 21 9 6 1...,10 21 1 11 22 20 21 1 20 9 16 24 20 1 21 9 6 1...
1,p271_100.wav,DIFF,they are no r devorced,there are no divorced,they are now divorced,21 9 6 26 1 2 19 6 1 15 16 1 19 1 5 6 23 16 19...,21 9 6 19 6 1 2 19 6 1 15 16 1 5 10 23 16 19 4...,21 9 6 26 1 2 19 6 1 15 16 24 1 5 10 23 16 19 ...
2,p271_134.wav,DIFF,nobody wil know the feling of the branches unt...,nobody wil know the feling of the brenches unt...,nobody will know the feeling of the branches u...,15 16 3 16 5 26 1 24 10 13 1 12 15 16 24 1 21 ...,15 16 3 16 5 26 1 24 10 13 1 12 15 16 24 1 21 ...,15 16 3 16 5 26 1 24 10 13 13 1 12 15 16 24 1 ...
3,p271_111.wav,DIFF,the cause of the fire is not yet knowne,the cause of the fires not yet known,the cause of the fire is not yet known,21 9 6 1 4 2 22 20 6 1 16 7 1 21 9 6 1 7 10 19...,21 9 6 1 4 2 22 20 6 1 16 7 1 21 9 6 1 7 10 19...,21 9 6 1 4 2 22 20 6 1 16 7 1 21 9 6 1 7 10 19...
4,p271_119.wav,DIFF,you have beter believe it,you had beter believe it,you had better believe it,26 16 22 1 9 2 23 6 1 3 6 21 6 19 1 3 6 13 10 ...,26 16 22 1 9 2 5 1 3 6 21 6 19 1 3 6 13 10 6 2...,26 16 22 1 9 2 5 1 3 6 21 21 6 19 1 3 6 13 10 ...
5,p271_108.wav,DIFF,we have an election in eight days,we have an election in a days,we have an election in eight days,24 6 1 9 2 23 6 1 2 15 1 6 13 6 4 21 10 16 15 ...,24 6 1 9 2 23 6 1 2 15 1 6 13 6 4 21 10 16 15 ...,24 6 1 9 2 23 6 1 2 15 1 6 13 6 4 21 10 16 15 ...
7,p271_135.wav,DIFF,it was very frighten and dirty,it was very frightened and dirty,it was very frightened and dirty,10 21 1 24 2 20 1 23 6 19 26 1 7 19 10 8 9 21 ...,10 21 1 24 2 20 1 23 6 19 26 1 7 19 10 8 9 21 ...,10 21 1 24 2 20 1 23 6 19 26 1 7 19 10 8 9 21 ...
9,p271_098.wav,DIFF,it is a nowing,it is a noine,it is annoying,10 21 1 10 20 1 2 1 15 16 24 10 15 8,10 21 1 10 20 1 2 1 15 16 10 15 6,10 21 1 10 20 1 2 15 15 16 26 10 15 8
10,p271_091.wav,DIFF,downing stret wil make the second apointment i...,doning stretwil make the second apointment in ...,downing street will make the second appointmen...,5 16 24 15 10 15 8 1 20 21 19 6 21 1 24 10 13 ...,5 16 15 10 15 8 1 20 21 19 6 21 24 10 13 1 14 ...,5 16 24 15 10 15 8 1 20 21 19 6 6 21 1 24 10 1...
11,p271_106.wav,DIFF,they make a selectiove perception proces,they make a selective perception proces,they make a selective perception process,21 9 6 26 1 14 2 12 6 1 2 1 20 6 13 6 4 21 10 ...,21 9 6 26 1 14 2 12 6 1 2 1 20 6 13 6 4 21 10 ...,21 9 6 26 1 14 2 12 6 1 2 1 20 6 13 6 4 21 10 ...


The original audio:

In [8]:
index = 15

filename = 'vctk/' + tpv10.ix[index]['filename']
print filename

# Load NN outputs

orig_mfcc = origv10[0][index].transpose([1, 0])
adv_mfcc = advv10[0][index].transpose([1, 0])
zero_padding = ~np.all(orig_mfcc == 0, axis=0)
orig_mfcc = orig_mfcc[:, zero_padding]
adv_mfcc = adv_mfcc[:, zero_padding]


vctk/p271_120.wav


The reconstructed original file:

In [16]:
imfcc.transform(orig_mfcc)
imfcc.play_transformed()
#imfcc.get_mfcc()

The reconstructed adversarial example:

In [10]:
imfcc.transform(adv_mfcc)
imfcc.play_transformed()
#imfcc.get_mfcc()

#### Effect of MFCC noise

In [11]:
imfcc.modify_parameters(mfcc_noise=20.0)
imfcc.transform(adv_mfcc)
imfcc.modify_parameters(mfcc_noise=0.0)
imfcc.play_transformed()

# Fooling Gradient

"I was rubbish" to "I was awesome"

In [12]:
orig = np.load("../rubbish.npy", allow_pickle=False)

imfcc.transform(orig)
imfcc.play_transformed()
#imfcc.get_mfcc()

In [13]:
adv = np.load("../awesome.npy", allow_pickle=False)[0,:,:].transpose([1,0])

imfcc.transform(adv)
imfcc.play_transformed()
#imfcc.get_mfcc()
#sum(sum(abs(orig-adv)))/sum(sum(abs(orig)))

"Please call Stella" to "Siri call police"

In [14]:
orig = np.load("../stella.npy", allow_pickle=False)

imfcc.transform(orig)
imfcc.play_transformed()

In [15]:
adv = np.load("../police.npy", allow_pickle=False)[0,:,:].transpose([1,0])

imfcc.transform(adv)
imfcc.play_transformed()
#imfcc.get_mfcc()
#sum(sum(abs(orig-adv)))/sum(sum(abs(orig)))